Distributed Meshes and Spaces
===

Setting up the client and the world-communiactor:

In [1]:
from ipyparallel import Cluster
c = await Cluster(engines="mpi").start_and_connect(n=4, activate=True)
c.ids

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/4 [00:00<?, ?engine/s]

[0, 1, 2, 3]

In [2]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD

The master generates a mesh, which is then distributed within the team of processors. The master process calls the graph partitioning library metis, which assigns a process id to each element. Then, elements are sent to the process with according rank.

The master process does not keep elements itself, it is kept free for special administrative work.

The distribution is done for the Netgen mesh. Parallel uniform refinement of the Netgen-mesh is also possible.

In [3]:
%%px
from ngsolve import *

if comm.rank == 0:
    ngmesh = unit_square.GenerateMesh(maxh=0.1)
    print ("global num els =", len(ngmesh.Elements2D()))
    ngmesh.Distribute(comm)
else:
    ngmesh = netgen.meshing.Mesh.Receive(comm)

for l in range(2):
    ngmesh.Refine()
    
mesh = Mesh(ngmesh)
print ("process", comm.rank, "got elements:", mesh.GetNE(VOL))

[stdout:0] global num els = 226
process 0 got elements: 0


[stdout:2] process 2 got elements: 1232


[stdout:3] process 3 got elements: 1184


[stdout:1] process 1 got elements: 1200


The collective communication `reduce` combines data from each process to one global value. Default reduction operation is summation. Only the root process gets the result. Alternatively, use 'allreduce' to broadcast the result to all team members:

In [4]:
%%px
sumup = comm.reduce(mesh.GetNE(VOL))
print ("summing up num els: ", sumup)

[stdout:2] summing up num els:  None


[stdout:0] summing up num els:  3616


[stdout:3] summing up num els:  None


[stdout:1] summing up num els:  None


We can retriev the `mesh` variable from each node of the cluster. The master process returns the global mesh, each worker only its local part. The list 'meshes' obtains the list of meshes.

In [5]:
from ngsolve import *
from ngsolve.webgui import Draw
meshes = c[:]['mesh']
for i,m in enumerate(meshes):
    print ("mesh of rank", i)
    Draw (m)

mesh of rank 0


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

mesh of rank 1


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

mesh of rank 2


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

mesh of rank 3


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

Distributed finite element spaces
===

We can define finite element spaces on the distributed mesh. 
Every process only defines dofs on its subset of elements:

In [6]:
%%px
fes = H1(mesh, order=2)
print ("ndof local =", fes.ndof, ", ndof global =", fes.ndofglobal)

[stdout:3] ndof local = 2465 , ndof global = 7393


[stdout:1] ndof local = 2509 , ndof global = 7393


[stdout:0] ndof local = 0 , ndof global = 7393


[stdout:2] ndof local = 2565 , ndof global = 7393


In [7]:
%%px
sumlocdofs = comm.reduce (fes.ndof)
if comm.rank == 0:
    print ("sum of local dofs:", sumlocdofs)

[stdout:0] sum of local dofs: 7539


The sum of local dofs is larger than the global number of dofs, since dofs at interface nodes are counted multiplel times.

We can define distributed grid-functions. Global operations like the `Integrate` function performs local integration, and sum up the result:

In [8]:
%%px
gfu = GridFunction(fes)
gfu.Set(x*y)
print ("integrate:", Integrate(gfu, mesh))

[stdout:1] integrate: 0.24999999999999922


[stdout:2] integrate: 0.24999999999999922


[stdout:3] integrate: 0.24999999999999922


[stdout:0] integrate: 0.24999999999999922


We can retrieve the gridfunction to the local Python scope:

In [9]:
gfus = c[:]['gfu']
print ("integrate locally:", Integrate(gfus[0], gfus[0].space.mesh))

Draw (gfus[0], min=0,max=1)
Draw (gfus[1], min=0,max=1);

integrate locally: 0.2499999999999994


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

We can use a piece-wise constant space to visualize the mesh partitioning:

In [11]:
%%px
gfl2 = GridFunction(L2(mesh, order=0))
gfl2.vec[:] = comm.rank

In [12]:
Draw (c[:]['gfl2'][0]);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

The `ParallelDofs` class
---

The back-bone of connection of dofs is the `ParallelDofs` class. It is provided by a distributed finite element space, based on the connectivity of the mesh. The pardofs object knows with which other processes the dof is shared. We can ask for all dof numbers shared with a particular process, and obtain a list of local dof nrs. The ordering is consistent for both partners. 

In [13]:
%%px
pardofs = fes.ParallelDofs()
for otherp in range(comm.size):
    print ("with process", otherp, "I share dofs", \
           list(pardofs.Proc2Dof(otherp)))

[stdout:0] with process 0 I share dofs []
with process 1 I share dofs []
with process 2 I share dofs []
with process 3 I share dofs []


[stdout:1] with process 0 I share dofs []
with process 1 I share dofs []
with process 2 I share dofs [6, 20, 32, 43, 48, 50, 51, 68, 104, 137, 164, 174, 177, 198, 256, 258, 324, 326, 384, 386, 412, 415, 424, 426, 429, 675, 733, 735, 801, 803, 861, 863, 889, 892, 901, 903, 906, 985, 986, 1179, 1180, 1373, 1374, 1531, 1532, 1589, 1590, 1605, 1606]
with process 3 I share dofs [5, 18, 19, 30, 41, 51, 66, 100, 101, 132, 160, 196, 250, 251, 254, 255, 314, 316, 374, 377, 427, 673, 727, 728, 731, 732, 791, 793, 851, 854, 904, 977, 978, 1159, 1160, 1163, 1164, 1345, 1346, 1509, 1510]


[stdout:2] with process 0 I share dofs []
with process 1 I share dofs [12, 23, 32, 39, 44, 46, 48, 52, 53, 54, 55, 56, 57, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 713, 772, 773, 822, 823, 859, 860, 884, 885, 894, 895, 903, 923, 924, 927, 928, 931, 932, 935, 936, 939, 940, 943, 944]
with process 2 I share dofs []
with process 3 I share dofs [1, 13, 24, 33, 34, 42, 48, 49, 61, 93, 121, 141, 143, 165, 175, 195, 239, 241, 298, 300, 346, 348, 349, 351, 394, 396, 417, 420, 421, 670, 715, 717, 776, 778, 826, 828, 829, 831, 876, 878, 904, 907, 908, 959, 960, 1131, 1132, 1297, 1298, 1415, 1416, 1425, 1426, 1555, 1556, 1613, 1614]


[stdout:3] with process 0 I share dofs []
with process 1 I share dofs [1, 13, 14, 26, 43, 47, 50, 51, 52, 53, 54, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 644, 689, 690, 691, 692, 760, 761, 854, 855, 874, 887, 888, 891, 892, 895, 896, 899, 900, 903, 904]
with process 2 I share dofs [12, 25, 35, 40, 41, 45, 47, 49, 55, 56, 57, 58, 59, 60, 61, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 687, 756, 757, 811, 812, 840, 841, 845, 846, 864, 865, 875, 884, 885, 907, 908, 911, 912, 915, 916, 919, 920, 923, 924, 927, 928, 931, 932]
with process 3 I share dofs []
